In [ ]:
!pip install huggingface_hub diffusers accelerate

In [ ]:
!git lfs install
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/ByteDance/AnimateDiff-Lightning

In [ ]:
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

device = "cuda"
dtype = torch.float16

In [15]:
step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "Lykon/AbsoluteReality"  # Choose to your favorite base model.

In [ ]:
adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo ,ckpt), device=device))
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")

In [ ]:
output = pipe(prompt="spongebob is friends with blue whale", guidance_scale=1.0, num_inference_steps=step)
export_to_gif(output.frames[0], "image.gif")

In [ ]:
from IPython.display import Image
Image(open('image.gif','rb').read())